# 딥러닝세션3주차 과제

7조 곽현지

## Q1

1. zero padding을 크기 1만큼 적용  
2. stride를 (1,1)로 filter A를 적용하고  
3. stride (2,2)로 2 * 2 maxpooling을 적용했을 때의 이미지 파일은 어떻게 생겼는지 예측

In [1]:
import numpy as np
import pandas as pd

In [2]:
# input image
image = np.array([[2,0,2,1,3,2],
                  [0,2,0,2,2,2],
                  [1,0,1,3,1,1],
                  [0,0,1,1,1,0],
                  [0,1,3,4,1,0],
                  [0,1,0,0,5,2]])

# filter
A = np.array ([[1,0,0],
               [0,1,0],
               [0,0,1]])

In [3]:
# zero padding
padding = np.pad(image, ((1,1),(1,1)), 'constant', constant_values=0)
padding

array([[0, 0, 0, 0, 0, 0, 0, 0],
       [0, 2, 0, 2, 1, 3, 2, 0],
       [0, 0, 2, 0, 2, 2, 2, 0],
       [0, 1, 0, 1, 3, 1, 1, 0],
       [0, 0, 0, 1, 1, 1, 0, 0],
       [0, 0, 1, 3, 4, 1, 0, 0],
       [0, 0, 1, 0, 0, 5, 2, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]])

In [4]:
# fiter A 적용(convolution)
output_size = (len(image)-len(A)+2*1)  + 1
conv = []

for i in range(output_size):
    for j in range(output_size):
        filtered = padding[i:i+3, j:j+3] * A
        conv.append(np.sum(filtered))

conv = np.array(conv).reshape(output_size, output_size)
conv

array([[ 4,  0,  4,  3,  5,  2],
       [ 0,  5,  3,  5,  4,  5],
       [ 1,  1,  4,  4,  3,  3],
       [ 1,  4,  5,  3,  4,  1],
       [ 1,  1,  3, 10,  4,  1],
       [ 0,  1,  1,  3,  9,  3]])

In [5]:
# maxpooling
stride = 2
output_size = int(output_size/stride)
pooling = []

for i in range(0, len(conv), stride):
    for j in range(0, len(conv), stride):
        pooling.append(np.max(conv[i:i+stride, j:j+stride]))

pooling = np.array(pooling).reshape(output_size, output_size)
pooling

array([[ 5,  5,  5],
       [ 4,  5,  4],
       [ 1, 10,  9]])

## Q2

1. layer1의 kernel size를 4로 늘리고, layer3의 stride를 2로 늘려라  
2. layer3의 output size를 구하여라  
3. cost와 accuracy가 기존CNN모델에 비해 어떻데 변할 것인가?  
4. 코딩하여 cost와 accuracy를 확인해보자  

In [6]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [7]:
# parameter 설정
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [8]:
# dataset 
mnist_train = dsets.MNIST(root = 'MNIST_data/',
                          train = True,
                          transform = transforms.ToTensor(),
                          download = False)

mnist_test = dsets.MNIST(root = 'MNIST_data/',
                         train = False,
                         transform = transforms.ToTensor(),
                         download = False)

In [9]:
#dataset loader
data_loader = torch.utils.data.DataLoader (dataset= mnist_train,
                                          batch_size = batch_size,
                                          shuffle =True,
                                          drop_last = True)

In [10]:
#기존 CNN model
class CNN(torch.nn.Module):
    
    def __init__(self):
        
        super(CNN,self).__init__()
        self.keep_prob = 0.5
        
        self.layer1 = torch.nn.Sequential(torch.nn.Conv2d(1,32,kernel_size=3, stride=1, padding=1),
                                         torch.nn.ReLU(), 
                                         torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer2 = torch.nn.Sequential(torch.nn.Conv2d(32,64,kernel_size=3, stride=1, padding=1),
                                         torch.nn.ReLU(), 
                                         torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer3 = torch.nn.Sequential(torch.nn.Conv2d(64,128, kernel_size=3, stride=1, padding=1),
                                         torch.nn.ReLU(), 
                                         torch.nn.MaxPool2d(kernel_size=2,stride=2, padding=1))
        
        self.fc1 = torch.nn.Linear(4*4*128,625, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        
        self.layer4 = torch.nn.Sequential(
            self.fc1, torch.nn.ReLU(),
            torch.nn.Dropout(p=1-self.keep_prob))
        
        self.fc2 = torch.nn.Linear(625, 10, bias =True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
   
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0),-1)
        out = self.layer4(out)
        out = self.fc2(out)
        return out

In [11]:
model = CNN()

In [12]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr= learning_rate)

In [13]:
total_batch = len(data_loader)
print("Learning started. It takes sometime.")
for epoch in range(training_epochs):
    avg_cost = 0
    
    for X,Y in data_loader:
        X=X
        Y=Y
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis,Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch+1, avg_cost))

print('Learning Finished')

Learning started. It takes sometime.
[Epoch:    1] cost = 0.196631297
[Epoch:    2] cost = 0.0542656519
[Epoch:    3] cost = 0.0380119011
[Epoch:    4] cost = 0.0289844479
[Epoch:    5] cost = 0.0226048175
[Epoch:    6] cost = 0.0204740018
[Epoch:    7] cost = 0.016136216
[Epoch:    8] cost = 0.0156537648
[Epoch:    9] cost = 0.0125661725
[Epoch:   10] cost = 0.0102963997
[Epoch:   11] cost = 0.0103794811
[Epoch:   12] cost = 0.00843036082
[Epoch:   13] cost = 0.010024826
[Epoch:   14] cost = 0.00619400758
[Epoch:   15] cost = 0.0080791954
Learning Finished


In [14]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float()
    Y_test =  mnist_test.test_labels
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

C:\Users\njj06\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\njj06\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9638000130653381


In [15]:
# 새로운 model
class CNN(torch.nn.Module):
    
    def __init__(self):
        
        super(CNN,self).__init__()
        self.keep_prob = 0.5
        
        self.layer1 = torch.nn.Sequential(torch.nn.Conv2d(1,32,kernel_size=4, stride=1, padding=1),
                                         torch.nn.ReLU(), 
                                         torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer2 = torch.nn.Sequential(torch.nn.Conv2d(32,64,kernel_size=3, stride=1, padding=1),
                                         torch.nn.ReLU(), 
                                         torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer3 = torch.nn.Sequential(torch.nn.Conv2d(64,128, kernel_size=3, stride=2, padding=1),
                                         torch.nn.ReLU(), 
                                         torch.nn.MaxPool2d(kernel_size=2,stride=2, padding=1))
        
        self.fc1 = torch.nn.Linear(2*2*128,625, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        
        self.layer4 = torch.nn.Sequential(
            self.fc1, torch.nn.ReLU(),
            torch.nn.Dropout(p=1-self.keep_prob))
        
        self.fc2 = torch.nn.Linear(625, 10, bias =True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
   
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0),-1)
        out = self.layer4(out)
        out = self.fc2(out)
        return out

In [16]:
model = CNN()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr= learning_rate)

In [17]:
total_batch = len(data_loader)
print("Learning started. It takes sometime.")
for epoch in range(training_epochs):
    avg_cost_ = 0
    
    for X,Y in data_loader:
        X=X
        Y=Y
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost_ = criterion(hypothesis,Y)
        cost_.backward()
        optimizer.step()
        
        avg_cost_ += cost_ / total_batch
        
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch+1, avg_cost_))

print('Learning Finished')

Learning started. It takes sometime.
[Epoch:    1] cost = 0.259248346
[Epoch:    2] cost = 0.0657888576
[Epoch:    3] cost = 0.0465329066
[Epoch:    4] cost = 0.0366148427
[Epoch:    5] cost = 0.0308353454
[Epoch:    6] cost = 0.0248770732
[Epoch:    7] cost = 0.0212437529
[Epoch:    8] cost = 0.0182358362
[Epoch:    9] cost = 0.0169584993
[Epoch:   10] cost = 0.0139618311
[Epoch:   11] cost = 0.0142764784
[Epoch:   12] cost = 0.0113195032
[Epoch:   13] cost = 0.010868324
[Epoch:   14] cost = 0.00950994715
[Epoch:   15] cost = 0.00976291392
Learning Finished


In [18]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float()
    Y_test =  mnist_test.test_labels
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy_ = correct_prediction.float().mean()
    print('Accuracy:', accuracy_.item())

Accuracy: 0.9860000014305115


In [22]:
print(avg_cost.item()); print(avg_cost_.item())
print(accuracy.item()); print(accuracy_.item())

0.008079195395112038
0.009762913919985294
0.9638000130653381
0.9860000014305115


2. (2, 2, 128)
3. cost는 약간 높아지고(유의미하지 않음) accuracy는 높아졌다.  
